In [1]:
# | default_exp frontend.client

In [2]:
# | export
import json
import os
import re
from openai import OpenAI
from rich.console import Console
from rich.markdown import Markdown
from rich.theme import Theme
from agentic.backend.tools.manager import ToolManager
from agentic.backend.tools.fs_read import FsReadParams
from agentic.backend.tools.fs_write import FsWriteParams
from agentic.backend.schemas import ToolCall,  ExecuteBashParams, IntrospectParams, TodoParams
from agentic.backend.llm_factory import LLMClient
from pydantic import ValidationError
from typing import List, Dict, Any, Optional

from agentic.configs.prompts import get_system_prompt
from agentic.configs.loader import get_model_config, get_settings_config, get_tools_config, get_reasoning_config
# from agentic.backend.tracing import get_tracer

In [3]:
# | export
RESET = "\033[0m"
SYSTEM_PROMPT = get_system_prompt()


# Define a custom theme that removes backgrounds
custom_theme = Theme({
    "markdown.code": "default",        # remove background from inline code
    "markdown.code_block": "default",  # remove background from code blocks
    "markdown.h1": "bold",             # keep bold but no background
    "markdown.h2": "bold",
    "markdown.h3": "bold",
})

In [4]:
print(SYSTEM_PROMPT)

You are Buddy, an autonomous AI assistant built for comprehensive software development.
            You are running under the `buddy chat` CLI command.
            
            ---
            
            # Identity & Mission
            - **Role:** Autonomous software development assistant
            - **Personality:** Precise, structured, and concise (avoid unnecessary verbosity)
            - **Goal:** Deliver correct, efficient, and safe solutions, while minimizing user effort
            - **Output:** Always use valid Markdown syntax (headers, bullet points, tables, code blocks)
            
            ---
            
            # Core Capabilities
            
            ## Intelligence
            - **Adaptive Task Planning** – Decide when to use `task_planner` based on request complexity
            - **Advanced File Operations** – 2-mode `fs_read` (discover for file listing/fuzzy search, extract for content with regex/context) with optimized performance
            - **C

In [5]:
# | export
class BuddyClient:
    def __init__(self, model=None, base_url=None, api_key=None):
        # Load config defaults
        settings_config = get_settings_config()
        
        # Initialize LLM client
        self.llm_client = LLMClient(model, base_url, api_key)
        
        # Store config settings
        self.auto_approve = settings_config.get('auto_approve', False)
        
        self.tool_manager = ToolManager()
        self.conversation_history = [{"role": "system", "content": SYSTEM_PROMPT}]  # Session history
        
        # Initialize console for rich markdown rendering
        self.console = Console(theme=custom_theme)
        
        # Initialize MLflow tracing
        # self.tracer = get_tracer()
        # self.tracer.start_session()
    
    def chat(self, message, tools=None, stream=None):
        """Enhanced chat with OpenAI tool calling and Pydantic validation"""
        if tools is None:
            tools_config = get_tools_config()
            tools = tools_config.get('default_tools', [
                "fs_read", "fs_write", "execute_bash", "code_interpreter", 
                "code_quality", "doc_generator", "memory_manager", 
                "introspect", "debate_agent", "task_planner", "task_monitor", "task_executor"
            ])
        
        if stream is None:
            settings_config = get_settings_config()
            stream = settings_config.get('stream', True)
        
        # Add user message to history
        self.conversation_history.append({"role": "user", "content": message})
        
        # Get retry count from reasoning config
        reasoning_config = get_reasoning_config()
        max_retries = reasoning_config.get('retry_count', 2)
        retry_count = 0
        result = None
        
        while retry_count <= max_retries:
            try:
                # Get OpenAI-formatted tools
                openai_tools = self.tool_manager.get_tools(tools)
                
                response = self.llm_client.create_completion(
                    messages=self.conversation_history,
                    tools=openai_tools,
                    stream=stream
                )
                
                # Store the response for tracing
                self._last_response = response
                
                if stream:
                    result = self.llm_client.handle_streaming_response(response, self.console)
                else:
                    result = self.llm_client.process_response(response,self.console)
                    
                # Execute tool calls if any
                if result.get("tool_calls"):
                    executed_calls = self._execute_tool_calls(result["tool_calls"])
                    result["tool_calls"] = executed_calls

                
                # Add assistant response to history
                assistant_message = {"role": "assistant", "content": result.get("content", "")}
                if result.get("tool_calls"):
                    assistant_message["tool_calls"] = result["tool_calls"]
                self.conversation_history.append(assistant_message)
                
                # If no tool calls, conversation is complete
                if not result.get("tool_calls"):
                    break
                
                # Add tool results to history
                for tool_call in result.get("tool_calls", []):
                    if hasattr(tool_call, 'get') and tool_call.get("result"):
                        self.conversation_history.append({
                            "role": "tool",
                            "tool_call_id": tool_call.get("id", ""),
                            "content": str(tool_call["result"])
                        })
                
            except Exception as e:
                retry_count += 1
                if retry_count <= max_retries:
                    print(f"⚠️ Error in conversation (attempt {retry_count}/{max_retries + 1}): {e}")
                    continue
                else:
                    print(f"⚠️ Error in conversation (final attempt): {e}")
                    break
        
        return result or {"content": "Error: Unable to complete request", "tool_calls": []}
    
    def clear_history(self):
        """Clear conversation history"""
        self.conversation_history = [{"role": "system", "content": SYSTEM_PROMPT}]
        print("🗑️ Conversation history cleared")
    
    def show_history(self):
        """Show conversation history"""
        print("\n📜 Conversation History:")
        for i, msg in enumerate(self.conversation_history[1:], 1):  # Skip system message
            role = msg["role"].upper()
            content = msg.get("content", "")[:100] + "..." if len(msg.get("content", "")) > 100 else msg.get("content", "")
            print(f"{i}. {role}: {content}")
        print()

    
    def _execute_tool_calls(self, tool_calls: List[Dict]):
        """Execute tool calls with Pydantic validation and user permission"""
        executed_calls = []
        
        for tool_call in tool_calls:
            try:
                function_name = tool_call["function"]["name"]
                arguments_str = tool_call["function"]["arguments"]
                
                # Parse arguments
                try:
                    arguments = json.loads(arguments_str)
                except json.JSONDecodeError as e:
                    print(f"\n⚠️ Invalid JSON in tool call: {e}")
                    print("🔄 Skipping this tool call...")
                    continue
                
                # Validate with Pydantic
                if not self._validate_tool_call(function_name, arguments):
                    print(f"⚠️ Skipping {function_name} due to validation errors")
                    continue
                
                # Show command and get permission
                if not self._get_permission(function_name, arguments):
                    print("❌ Command cancelled")
                    continue
                
                # Execute the tool
                result = self.tool_manager.execute_tool(function_name, arguments)
                
                # Format and display result
                formatted_result = self._format_tool_result(function_name, result)
                print(f"✅ {formatted_result}")
                
                # Store result for conversation continuity
                tool_call["result"] = result
                executed_calls.append(tool_call)
                
            except Exception as e:
                print(f"\n⚠️ Tool execution error: {e}")
                print("🔄 Skipping this tool call...")
                continue
        
        return executed_calls
    
    def _get_permission(self, function_name: str, arguments: Dict[str, Any]) -> bool:
        """Get user permission before executing commands"""
        if self.auto_approve:
            # Show beautiful tool display even with auto-approve
            self._display_tool_call(function_name, arguments, trusted=True)
            return True
        
        # Show beautiful tool display
        self._display_tool_call(function_name, arguments, trusted=False)
        
        while True:
            response = input("Execute? (y)es/(n)o/(t)rust always: ").lower().strip()
            if response in ['y', 'yes']:
                return True
            elif response in ['n', 'no']:
                return False
            elif response in ['t', 'trust']:
                self.auto_approve = True
                print("✅ Auto-approval enabled for this session")
                return True
            else:
                print("Please enter 'y', 'n', or 't'")
    
    def _display_tool_call(self, function_name: str, arguments: Dict[str, Any], trusted: bool = False):
        """Display tool call in Amazon Q style"""
        trust_text = " (trusted)" if trusted else ""
        description = self._get_command_description(function_name, arguments)
        command_preview = self._get_command_preview(function_name, arguments)
        
        print(f"\n🛠️  Using tool: {function_name}{trust_text}")
        print(" ⋮")
        print(" ● I will run the following:")
        print(f"{command_preview}")
        print(" ⋮")
        print(f" ↳ Purpose: {description}")
        if not trusted:
            print()  # Extra line before prompt
    
    def _get_command_description(self, function_name: str, arguments: Dict[str, Any]) -> str:
        """Generate one-sentence description of what the command does"""
        if function_name == "fs_read":
            ops = arguments.get("operations", [])
            if ops and ops[0].get("mode") == "discover":
                return "Lists files in a directory by query and pattern."
            elif ops and ops[0].get("mode") == "extract":
                return "Extracts content from files matching a query."
        elif function_name == "fs_write":
            cmd = arguments.get("command")
            if cmd == "create":
                return "Creates a new file with the specified content"
            elif cmd == "str_replace":
                return "Replaces text in an existing file"
            elif cmd == "append":
                return "Adds content to the end of an existing file"
        elif function_name == "execute_bash":
            return f"Runs the bash command: {arguments.get('command')}"
        elif function_name == "code_interpreter":
            return f"Executes Python code: {arguments.get('code', '')[:50]}..."
        elif function_name == "code_quality":
            return f"Analyzes code quality for repository: {arguments.get('repo_path')}"
        elif function_name == "doc_generator":
            return f"Generates documentation for: {arguments.get('repo_path')}"
        elif function_name == "memory_manager":
            return f"Manages conversation memory: {arguments.get('action')}"
        elif function_name == "introspect":
            return f"Self-analysis: {arguments.get('action')}"
        elif function_name == "debate_agent":
            return f"Multi-perspective analysis of: {arguments.get('decision')}"
        elif function_name == "todo":
            action = arguments.get("action")
            if action == "plan":
                return "Breaks down a complex task into smaller steps"
            elif action == "execute":
                return "Executes the next step in the task plan"
        
        return "Executes the specified operation"
    
    def _get_command_preview(self, function_name: str, arguments: Dict[str, Any]) -> str:
        """Generate a preview of the actual command"""
        if function_name == "fs_read":
            ops = arguments.get("operations", [])
            if ops:
                op = ops[0]
                if op.get("mode") == "Directory":
                    return f"ls {op.get('path', '.')}"
                elif op.get("mode") == "Line":
                    return f"cat {op.get('path')}"
                elif op.get("mode") == "Search":
                    return f"grep '{op.get('pattern')}' {op.get('path')}"
        elif function_name == "fs_write":
            cmd = arguments.get("command")
            path = arguments.get("path")
            if cmd == "create":
                return f"echo 'content' > {path}"
            elif cmd == "str_replace":
                return f"sed -i 's/old/new/g' {path}"
            elif cmd == "append":
                return f"echo 'content' >> {path}"
        elif function_name == "execute_bash":
            return arguments.get("command", "")
        elif function_name == "code_interpreter":
            return f"python -c \"{arguments.get('code', '')[:30]}...\""
        elif function_name == "todo":
            return f"todo {arguments.get('action')} '{arguments.get('task')}'"
        elif function_name == "introspect":
            return "buddy --introspect"
        
        return f"{function_name}({', '.join(f'{k}={v}' for k, v in arguments.items())})"
    
    def _validate_tool_call(self, function_name: str, arguments: Dict[str, Any]) -> bool:
        """Validate tool call parameters with Pydantic"""
        try:
            if function_name == "fs_read":
                FsReadParams(**arguments)
            elif function_name == "fs_write":
                FsWriteParams(**arguments)
            elif function_name == "execute_bash":
                ExecuteBashParams(**arguments)
            elif function_name == "introspect":
                IntrospectParams(**arguments)
            elif function_name == "todo":
                TodoParams(**arguments)
            else:
                # For new tools, skip validation for now
                return True
            
            return True
            
        except ValidationError as e:
            error_msg = str(e)
            if "Input should be 'Line', 'Directory' or 'Search'" in error_msg:
                print(f"\n⚠️ Invalid mode for {function_name}. Use 'Line' to read files, 'Directory' to list directories, 'Search' to find patterns.")
            else:
                print(f"\n⚠️ Validation error for {function_name}: {e}")
            return False
        except Exception as e:
            print(f"\n⚠️ Unexpected validation error for {function_name}: {e}")
            return False
    
    def _format_tool_result(self, function_name: str, result: Dict[str, Any]) -> str:
        """Format tool results for display"""
        if "error" in result:
            return f"Error: {result['error']}"
        
        if function_name == "fs_read":
            if "results" in result:
                formatted = []
                for res in result["results"]:
                    if "items" in res:
                        items = res["items"]
                        file_count = sum(1 for item in items if item.get('type') == 'file')
                        dir_count = sum(1 for item in items if item.get('type') == 'directory')
                        formatted.append(f"Directory {res['path']}: {file_count} files, {dir_count} directories")
                    elif "content" in res:
                        lines = len(res["content"].split('\n'))
                        formatted.append(f"File {res['path']}: {lines} lines")
                    elif "matches" in res:
                        match_count = len(res["matches"])
                        formatted.append(f"Search in {res['path']}: {match_count} matches")
                return "; ".join(formatted)
        
        elif function_name == "execute_bash":
            if "stdout" in result:
                output = result["stdout"].strip()
                return f"Exit {result.get('exit_status', 0)}: {output[:100]}{'...' if len(output) > 100 else ''}"
        
        elif function_name == "fs_write":
            if "success" in result:
                return result["message"]
        
        elif function_name == "code_interpreter":
            if "success" in result:
                plots = f", {result['plot_count']} plots" if result.get('plot_count', 0) > 0 else ""
                return f"Code executed in {result['execution_time']}s{plots}"
        
        elif function_name == "code_quality":
            if "summary" in result:
                return f"Quality analysis: {result['summary'].get('quality_score', 0)}/100 ({result['summary'].get('grade', 'N/A')})"
        
        elif function_name == "todo":
            if "steps" in result:
                return f"Created plan with {len(result['steps'])} steps"
            elif "step" in result:
                return f"Executed step: {result['step']['description']}"
        
        return str(result)


In [ ]:
# | export
def main(model=None, base_url=None, api_key=None, no_stream=False):
    """Enhanced interface with OpenAI tool calling (adapted for Jupyter)"""
    
    # Load config defaults
    model_config = get_model_config()
    
    # Use config values as defaults
    if model is None:
        model = model_config.get('name', 'qwen3:8b')
    if base_url is None:
        base_url = model_config.get('url', 'http://localhost:11434/v1')
    if api_key is None:
        api_key = model_config.get('api_key', 'ollama')
    
    # Create a simple object with attributes similar to argparse.Namespace
    class Args:
        pass
    
    args = Args()
    args.model = model
    args.base_url = base_url
    args.api_key = api_key
    args.no_stream = no_stream

    # Now create the client
    client = BuddyClient(
        model=args.model,
        base_url=args.base_url,
        api_key=args.api_key
    )
    
    # Enhanced welcome message using Rich
    from rich.console import Console
    from rich.panel import Panel
    from rich.text import Text
    from rich.table import Table
    
    console = Console(theme=custom_theme)
    
    # Create welcome header
    welcome_text = Text()
    welcome_text.append("🤖 Buddy CLI", style="bold cyan")
    welcome_text.append(" - Advanced Autonomous AI Assistant", style="bright_white")
    
    # Model info
    model_text = Text()
    model_text.append("🧠 Model: ", style="bold white")
    model_text.append(f"{args.model}", style="bright_green")
    if args.base_url and args.base_url != "https://api.openai.com/v1":
        model_text.append(f" ({args.base_url})", style="dim white")
    
    # Create tools table
    tools_table = Table(show_header=False, box=None, padding=(0, 1))
    tools_table.add_column("Category", style="bold")
    tools_table.add_column("Tools", style="green")
    
    tools_table.add_row("🛠️  Core Tools:", "fs_read, fs_write, execute_bash")
    tools_table.add_row("🧪 Analysis:", "code_interpreter, code_quality, analyzer")
    tools_table.add_row("🧠 Intelligence:", "introspect, debate_agent, memory_manager")
    tools_table.add_row("📋 Planning:", "task_planner, task_executor, task_monitor")
    tools_table.add_row("📚 Utilities:", "doc_generator, todo")
    
    # Commands
    commands_text = Text()
    commands_text.append("📋 Commands: ", style="bold red")
    commands_text.append("/clear, /history, /quit, /exit", style="white")
    
    # Create panel content
    panel_content = Text()
    panel_content.append_text(welcome_text)
    panel_content.append("\n")
    panel_content.append_text(model_text)
    panel_content.append("\n\n")
    
    # Add tools table to console buffer
    console.print(Panel(panel_content, title="✨ Welcome to Buddy ✨", border_style="bright_blue", padding=(1, 2)))
    console.print(tools_table)
    console.print()
    
    # Commands panel
    commands_panel = Panel(commands_text, title="💡 Quick Commands", border_style="yellow", padding=(0, 1))
    console.print(commands_panel)
    console.print("\n💬 [bold green]Ready to assist! Type your message or command:[/bold green]\n")
    
    while True:
        try:
            user_input = input(">> ").strip()
            
            if user_input.lower() in ['quit', 'exit', '/quit', '/exit']:
                break
            elif user_input.lower() in ['clear','/clear']:
                client.clear_history()
                continue
            elif user_input.lower() in ['/history']:
                client.show_history()
                continue
                
            if user_input:
                client.chat(user_input, stream=not args.no_stream)
                print()
                
        except KeyboardInterrupt:
            print("\nGoodbye!")
            break

if __name__ == "__main__":
    main()


╭──────────────────────────────────────────── ✨ Welcome to Buddy ✨ ─────────────────────────────────────────────╮
│                                                                                                                 │
│  🤖 Buddy CLI - Advanced Autonomous AI Assistant                                                                │
│  🧠 Model: gpt-oss:20b (http://192.168.29.147:11500/v1)                                                         │
│                                                                                                                 │
│                                                                                                                 │
│                                                                                                                 │
╰─────────────────────────────────────────────────────────────────────────────────────────────────────────────────╯

 🛠️  Core Tools:    fs_read, fs_write, execute_bash           
 🧪 Analysis:      code_interpreter, code_quality, analyzer  
 🧠 Intelligence:  introspect, debate_agent, memory_manager  
 📋 Planning:      task_planner, task_executor, task_monitor 
 📚 Utilities:     doc_generator, todo                       

╭─────────────────────────────────────────────── 💡 Quick Commands ───────────────────────────────────────────────╮
│ 📋 Commands: /clear, /history, /quit, /exit                                                                     │
╰─────────────────────────────────────────────────────────────────────────────────────────────────────────────────╯

💬 Ready to assist! Type your message or command:

>>  Hello


...


╭─────────────────────── 🤔 Thinking ───────────────────────╮
│ We need to read README.md. Directory path given is /home/pranav-pc/projects/applied-GenAI-lab/nbs/buddy/frontend. Search for README.md. Use fs_read discover? Probably there's a README.md in root. Use fs_read discover.
⚠️ Validation error for fs_read: 1 validation error for FsReadParams
operations.0.path
  Field required [type=missing, input_value={'max_results': 10, 'mode... 'pattern': 'README.md'}, input_type=dict]
    For further information visit https://errors.pydantic.dev/2.11/v/missing
⚠️ Skipping fs_read due to validation errors



>>  what validation error you have encountered ?


INFO:httpx:HTTP Request: POST http://192.168.29.147:11500/v1/chat/completions "HTTP/1.1 200 OK"



╭─────────────────────── 🤔 Thinking ───────────────────────╮
│ We need to read README.md. It's likely in the repo. Use fs_read discover mode.


⚠️ Validation error for fs_read: 1 validation error for FsReadParams
operations.0.path
  Field required [type=missing, input_value={'max_results': 10, 'mode... 'pattern': 'README.md'}, input_type=dict]
    For further information visit https://errors.pydantic.dev/2.11/v/missing
⚠️ Skipping fs_read due to validation errors

